In [2]:
import random
#this is done to make the dataset format into farseq which is binary because helsinky is trained on farseq format
def read_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        return file.readlines()

def write_file(filepath, lines):
    with open(filepath, 'w', encoding='utf-8') as file:
        for line in lines:
            file.write(line)

def split_dataset(en_lines, ml_lines, train_ratio=0.8, valid_ratio=0.2):
    assert len(en_lines) == len(ml_lines), "The number of lines in the two files should be the same"

    data = list(zip(en_lines, ml_lines))
    random.shuffle(data)

    train_size = int(train_ratio * len(data))
    valid_size = int(valid_ratio * len(data))

    train_data = data[:train_size]
    valid_data = data[train_size:]


    return train_data, valid_data

# File paths
english_file = 'ELRC_2922.en-ml.en'
malayalam_file = 'ELRC_2922.en-ml.ml'

# Read the files
en_lines = read_file(english_file)
ml_lines = read_file(malayalam_file)

# Split the dataset
train_data, valid_data = split_dataset(en_lines, ml_lines)

# Write the splits to files
write_file('train.en', [line[0] for line in train_data])
write_file('train.ml', [line[1] for line in train_data])
write_file('valid.en', [line[0] for line in valid_data])
write_file('valid.ml', [line[1] for line in valid_data])


print("Dataset successfully split into train, validation set")

Dataset successfully split into train, validation set


## Load the Dataset

In [3]:
def load_data(source_file, target_file):
    with open(source_file, 'r', encoding='utf-8') as src_f, open(target_file, 'r', encoding='utf-8') as tgt_f:
        sources = src_f.readlines()
        targets = tgt_f.readlines()
    
    assert len(sources) == len(targets), "Source and target files must have the same number of lines"
    return sources, targets
    


train_dataset = load_data('train.en', 'train.ml')
eval_dataset = load_data('valid.en', 'valid.ml')

In [10]:
len(train_dataset)
#len(eval_dataset)

2

## Tokenize the data

In [4]:
from transformers import AutoTokenizer
from datasets import Dataset
model_repo = 'Helsinki-NLP/opus-mt-en-ml'
tokenizer = AutoTokenizer.from_pretrained(model_repo)

def tokenize_function(examples):
    source = tokenizer(examples['source'], padding='max_length', truncation=True, max_length=128)
    target = tokenizer(examples['target'], padding='max_length', truncation=True, max_length=128)
    
    # Rename target inputs to 'labels' (which is expected by most seq-to-seq models)
    source['labels'] = target['input_ids'] #Input to the Model (input_ids): The model takes the tokenized source text as input to generate predictions.
    #Correct Output (labels): The model is also provided with the tokenized target text as the correct answer. During training, it compares its predictions against these labels to learn how to improve.
    
    return source
# Convert tuples of lists to datasets
sources, targets = train_dataset[0], train_dataset[1]
sources_eval, targets_eval = eval_dataset[0], eval_dataset[1]
train_dataset = Dataset.from_dict({'source': sources, 'target': targets}) # we r converting into datasets format because to use the map function below
eval_dataset = Dataset.from_dict({'source': sources_eval, 'target': targets_eval})

# Apply the tokenizer to the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

c:\Users\midhu\anaconda3\envs\pytorch_env\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/360 [00:00<?, ? examples/s]

Map:   0%|          | 0/91 [00:00<?, ? examples/s]

## Prepare the Dataset for the Trainer

In [5]:
# Remove the text columns to avoid errors during training
train_dataset = train_dataset.remove_columns(['source', 'target'])
eval_dataset = eval_dataset.remove_columns(['source', 'target'])
#removing columns is a form of preprocessing before the dataset is tokenized which means convert the text into numerical representations(input_ids, attentionmasks and labels)

# Set the format to PyTorch tensors
train_dataset.set_format('torch')
eval_dataset.set_format('torch')

## Pass the Datasets to the Trainer

In [6]:
from transformers import Trainer, TrainingArguments, AutoModelForSeq2SeqLM


training_args = TrainingArguments(
    output_dir='./results',        
    num_train_epochs=3,
    fp16 = True, #by default training arguments using fp32 which makes the model size large so changing it into fp16            
    per_device_train_batch_size=8,  
    per_device_eval_batch_size=8,   
    warmup_steps=500,               
    weight_decay=0.01,              
    logging_dir='./logs',           
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,  # Load the best model found at the end of training
    metric_for_best_model="eval_loss",  # Metric to use to select the best model
    greater_is_better=False,      # Set to True if a higher metric value is better, False for lower (e.g., for loss)
)

trainer = Trainer(
    model= AutoModelForSeq2SeqLM.from_pretrained(model_repo),                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=eval_dataset             
)

trainer.train()
# Save the tokenizer manually after training
tokenizer.save_pretrained('./results')

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit e

In [1]:
import transformers
import torch
import accelerate

print(transformers.__version__)
print(torch.__version__)
print(accelerate.__version__)

4.44.0
2.3.1
0.31.0


In [1]:
import PIL
print("Pillow version:", PIL.__version__)
import sys
print(sys.executable)

Pillow version: 10.4.0
c:\Users\midhu\anaconda3\envs\pytorch_env\python.exe


In [12]:
%pip install --upgrade pillow

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [ ]:
# Access the logs
logs = trainer.state.log_history
print(logs)

### Extract and Separate the Training and Evaluation Loss

In [ ]:
import matplotlib.pyplot as plt

# Extract training and evaluation losses
training_loss = []
eval_loss = []
steps = []

for log in logs:
    if 'loss' in log:
        training_loss.append(log['loss'])
        steps.append(log['step'])  # This captures the step at which the log was recorded
    if 'eval_loss' in log:
        eval_loss.append(log['eval_loss'])

# Check if losses have been correctly extracted
print(f'Training loss: {training_loss}')
print(f'Evaluation loss: {eval_loss}')

Plot the Losses

In [ ]:
# Plot Training Loss
plt.figure(figsize=(10, 6))
plt.plot(steps, training_loss, label="Training Loss")

# Plot Evaluation Loss - since there are only 3 points, plot them against the appropriate step
# Assuming eval_loss corresponds to the end of each epoch (steps 50, 100, 130 for example)
eval_steps = [50, 100, 130]
plt.plot(eval_steps, eval_loss, label="Evaluation Loss", linestyle='--', marker='o')

plt.xlabel("Steps")
plt.ylabel("Loss")
plt.title("Training and Evaluation Loss Over Time")
plt.legend()
plt.grid()
plt.show()

In [ ]:
#to save a file or folder in zip or any file type use shutil
# import shutil
# shutil.make_archive("results", "zip", "/content/results")

In [ ]:
from transformers import MarianMTModel, MarianConfig

# Load pretrained model
pretrained_model = MarianMTModel.from_pretrained(model_repo)

# Load fine-tuned model (or initialize from checkpoint)
finetuned_model = MarianMTModel.from_pretrained("/content/results/checkpoint-90")

# Inspect positional embeddings
print(finetuned_model.model.encoder.embed_positions.weight)
print(finetuned_model.model.decoder.embed_positions.weight)

# If needed, reinitialize or copy embeddings
finetuned_model.model.encoder.embed_positions.weight = pretrained_model.model.encoder.embed_positions.weight
finetuned_model.model.decoder.embed_positions.weight = pretrained_model.model.decoder.embed_positions.weight

In [ ]:
import shutil
import os

# Specify the path to the folder you want to delete
folder_path = '/content/results'

# Check if the folder exists
if os.path.exists(folder_path):
    # Delete the folder and all its contents
    shutil.rmtree(folder_path)
    print(f"Folder '{folder_path}' has been deleted.")
else:
    print(f"Folder '{folder_path}' does not exist.")

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

# Path to your TensorFlow event file
file_path = "/content/logs/events.out.tfevents.1724420738.451d604ea9e8.827.0"

# Initialize lists for storing extracted data
training_loss = []
eval_loss = []
steps = []
eval_steps = []

# Using TFRecordDataset to read the event file
dataset = tf.data.TFRecordDataset(file_path)

# Function to parse the event
def parse_event(event):
    for value in event.summary.value:
        if value.tag == 'train/loss':  # Use 'train/loss' as the tag for training loss
            training_loss.append(value.simple_value)
            steps.append(event.step)
        elif value.tag == 'eval/loss':  # Use 'eval/loss' as the tag for evaluation loss
            eval_loss.append(value.simple_value)
            eval_steps.append(event.step)  # Make sure to capture the step for evaluation loss

# Iterate over the dataset to extract data
for raw_record in dataset:
    event = tf.compat.v1.Event.FromString(raw_record.numpy())
    parse_event(event)

# Print extracted values for debugging
print(f'Training loss: {training_loss}')
print(f'Steps: {steps}')
print(f'Evaluation loss: {eval_loss}')
print(f'Evaluation steps: {eval_steps}')

# Plot the Training Loss
plt.figure(figsize=(10, 6))
plt.plot(steps, training_loss, label="Training Loss")

# Plot the Evaluation Loss
plt.plot(eval_steps, eval_loss, label="Evaluation Loss", linestyle='--', marker='o')

plt.xlabel("Steps")
plt.ylabel("Loss")
plt.title("Training and Evaluation Loss Over Time")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Initialize a set to store all tags for debugging
all_tags = set()

# Using TFRecordDataset to read the event file
dataset = tf.data.TFRecordDataset(file_path)

# Iterate over the dataset to extract all tags
for raw_record in dataset:
    event = tf.compat.v1.Event.FromString(raw_record.numpy())
    for value in event.summary.value:
        all_tags.add(value.tag)

# Print all available tags
print(f'Available tags in the event file: {all_tags}')